# Elmy Data Challenge ENS 
## Feature Engineering

In [208]:
import os
from typing import Tuple, Any

import holidays

import pandas as pd
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt
from pandas import Series, DataFrame

In [209]:
path_file_x_train = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))+ "/datasets/elmy_spot_vs_intraday/raw_data/X_train.csv"
path_file_y_train = os.path.abspath(os.path.join(os.getcwd(), "..", "..")) + "/datasets/elmy_spot_vs_intraday/raw_data/y_train.csv"
path_file_x_test = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))+  "/datasets/elmy_spot_vs_intraday/raw_data/X_test.csv"

path_param_models = os.path.dirname(os.getcwd()) + "/datasets/elmy_spot_vs_intraday/params_models/"

X_train = pd.read_csv(path_file_x_train)
X_test = pd.read_csv(path_file_x_test)
y_train = pd.read_csv(path_file_y_train)

X_train['DELIVERY_START'] = pd.to_datetime(X_train['DELIVERY_START'],utc = True)
X_test["DELIVERY_START"] = pd.to_datetime(X_test["DELIVERY_START"],utc = True)
y_train['DELIVERY_START'] = pd.to_datetime(y_train['DELIVERY_START'],utc = True)

In [210]:
def create_time_features(X: pd.DataFrame) -> pd.DataFrame:
    
    french_holidays = holidays.FR()
    X_updated = X.copy(deep = True)
    X_updated["hour"] = X_updated["DELIVERY_START"].dt.hour
    X_updated["day_of_week"] = X_updated["DELIVERY_START"].dt.day_of_week
    X_updated["day_of_month"] = X_updated["DELIVERY_START"].dt.day
    X_updated["month"] = X_updated["DELIVERY_START"].dt.month
    X_updated["is_weekend_or_holiday"] = X_updated["DELIVERY_START"].apply(lambda x: x in french_holidays or x.weekday() >= 5)
    
    X_updated["is_weekend_or_holiday"] = X_updated["is_weekend_or_holiday"].astype(int)
    
    return X_updated

In [211]:
def create_aggregation_features(
        X_train: pd.DataFrame,
        X_test : pd.DataFrame,
        feature_names : list[str],
        rolling_n_days : list[int]
    ) -> tuple[pd.DataFrame, pd.DataFrame]:
    
    """
    Computes aggregation features for all features in feature_names and roll them for all values in rolling_n_days
        
    Parameters:
    ----------
    X_train : pd.DataFrame
        pandas dataframe containing all features and indexed by delivery_start
    
    X_test : pd.DataFrame
        pandas dataframe containing all features and indexed by delivery_start 
    
    feature_names : list[str]
        list of features to aggregate 
        e.g : ['load_forecast','solar_forecast', 'nucelar_power_available']
    
    rolling_n_days : list[int]
        list of rolling days to aggregate features for
        e.g : [7,14,21]
        
    Returns
    ----------
    X_updated : pd.DataFrame 
    """
    
    X_train_temp = X_train.copy()
    X_train_temp['set'] = 'train'
    X_test_temp = X_test.copy()
    X_test_temp['set'] = 'test'
    
    df = pd.concat([X_train_temp,X_test_temp], axis = 0)
    df.set_index('DELIVERY_START', inplace = True, drop = True)
    
    for feature_name in feature_names:
        for nb_rolling in rolling_n_days: 
            rolling_feature_name = "rolling::" + f"avg_" + feature_name + f"__lag_{nb_rolling}_days"
            df[rolling_feature_name] = (
                df.groupby(['hour'])[feature_name].rolling(
                    window = f"{nb_rolling}D",
                    min_periods = 1,
                    closed = 'left',
                    on = df.index
                ).mean().reset_index(level = 0, drop = True)
            )
            
    df.reset_index(inplace = True)
    
    X_train_updated = df[df.set == 'train'].copy(deep = True)
    X_test_updated = df[df.set == 'test'].copy(deep = True)
    
    X_train_updated.drop(['set'], axis = 1, inplace = True)
    X_test_updated.drop(['set'], axis = 1, inplace = True)
    
    return X_train_updated,X_test_updated

In [212]:
def create_lagged_features(
        X_train: pd.DataFrame,
        X_test : pd.DataFrame,
        feature_names : list[str],
        hourly_lags : list[int]
    ) -> tuple[pd.DataFrame, pd.DataFrame]:
    
    X_train_temp = X_train.copy()
    X_train_temp['set'] = 'train'
    X_test_temp = X_test.copy()
    X_test_temp['set'] = 'test'
    
    df = pd.concat([X_train_temp,X_test_temp], axis = 0)
    df.set_index('DELIVERY_START', inplace = True, drop = True) 
    
    for feature_name in feature_names:
        for lagg in hourly_lags:
            lagged_feature_name = "lagged::" + feature_name + f"__lag_{lagg}_hour"
            df[lagged_feature_name] = df[feature_name].shift(lagg, freq = '1h')
    
    df.reset_index(inplace = True)
    
    X_train_updated = df[df.set == 'train'].copy(deep = True)
    X_test_updated = df[df.set == 'test'].copy(deep = True)

    X_train_updated.drop(['set'], axis = 1, inplace = True)
    X_test_updated.drop(['set'], axis = 1, inplace = True)
    
    return X_train_updated,X_test_updated

In [213]:
nb_rolling_days = [7,14,21]
features_to_agg = [
    'load_forecast',
    'coal_power_available',
    'gas_power_available',
    'nucelear_power_available',
    'wind_power_forecasts_average',
    'solar_power_forecasts_average'
]

lags_in_hours = [1,2,3]
features_to_lag = [
   'load_forecast',
    'coal_power_available',
    'gas_power_available',
    'nucelear_power_available',
    'wind_power_forecasts_average',
    'solar_power_forecasts_average' 
]

# Create CSV

In [215]:
X_train = create_time_features(X_train)
X_test = create_time_features(X_test)

X_train,X_test = create_lagged_features(X_train, X_test, features_to_agg, lags_in_hours)
X_train,X_test = create_aggregation_features(X_train, X_test, features_to_agg, nb_rolling_days)

In [216]:
X_test

,DELIVERY_START,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price,...,rolling::avg_gas_power_available__lag_21_days,rolling::avg_nucelear_power_available__lag_7_days,rolling::avg_nucelear_power_available__lag_14_days,rolling::avg_nucelear_power_available__lag_21_days,rolling::avg_wind_power_forecasts_average__lag_7_days,rolling::avg_wind_power_forecasts_average__lag_14_days,rolling::avg_wind_power_forecasts_average__lag_21_days,rolling::avg_solar_power_forecasts_average__lag_7_days,rolling::avg_solar_power_forecasts_average__lag_14_days,rolling::avg_solar_power_forecasts_average__lag_21_days
10605,2023-04-01 22:00:00+00:00,45814.0,3386.0,10902.0,36705.0,6359.0,0.0,56.764535,0.0,NaN,...,11887.833333,37547.500000,38006.363636,38268.888889,5392.000000,6751.636364,7003.388889,0.000000,0.000000,0.000000
10606,2023-04-01 23:00:00+00:00,44084.0,3386.0,10902.0,36705.0,6469.0,0.0,54.262133,0.0,NaN,...,11804.055556,37215.000000,37725.909091,37925.277778,5341.500000,6793.272727,7080.555556,0.000000,0.000000,0.000000
10607,2023-04-02 00:00:00+00:00,43281.0,3386.0,10902.0,36705.0,6511.0,0.0,78.105928,0.0,NaN,...,11829.888889,36887.500000,37606.818182,37852.500000,5231.750000,6746.272727,7097.888889,0.000000,0.000000,0.000000
10608,2023-04-02 01:00:00+00:00,40825.0,3386.0,10902.0,36705.0,6628.0,0.0,78.187557,0.0,NaN,...,11829.888889,36957.500000,37753.181818,37880.833333,5185.500000,6691.727273,7103.944444,0.000000,0.000000,0.000000
10609,2023-04-02 02:00:00+00:00,39181.0,3386.0,10902.0,36705.0,6700.0,0.0,96.765484,0.0,NaN,...,11829.888889,36957.500000,37753.181818,37880.833333,5164.000000,6607.000000,7090.166667,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15542,2023-10-24 17:00:00+00:00,49686.0,2226.0,11749.0,42980.0,4901.0,0.0,247.887323,0.0,125.67,...,11132.523810,42537.857143,42558.928571,42141.619048,7579.285714,6250.928571,4756.904762,115.714286,178.214286,247.285714
15543,2023-10-24 18:00:00+00:00,53397.0,2226.0,11749.0,42980.0,5584.0,0.0,343.192642,0.0,139.58,...,11134.571429,42683.571429,42623.214286,42157.380952,7799.428571,6465.071429,4900.761905,0.000000,0.000000,0.000000
15544,2023-10-24 19:00:00+00:00,50586.0,2226.0,11749.0,42980.0,6306.0,0.0,471.875973,0.0,147.93,...,11152.904762,42646.428571,42604.642857,42160.952381,7915.142857,6590.357143,4982.714286,0.000000,0.000000,0.000000
15545,2023-10-24 20:00:00+00:00,46777.0,2226.0,11749.0,42980.0,6959.0,0.0,595.528100,0.0,122.20,...,11143.238095,42646.428571,42604.642857,42179.047619,7991.428571,6651.928571,5021.666667,0.000000,0.000000,0.000000


In [217]:
pathfile_x_train = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))+ "/datasets/elmy_spot_vs_intraday/processed_data/second_preprocess/" + "X_train_second_preprocess.csv"
pathfile_x_test = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))+ "/datasets/elmy_spot_vs_intraday/processed_data/second_preprocess/" + "X_test_second_preprocess.csv"

In [218]:
#pathfile_x_train_first_preprocess = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))+ "/datasets/elmy_spot_vs_intraday/processed_data/first_preprocess/" + "X_train_first_preprocess.csv"
#pathfile_x_test_first_preprocess = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))+ "/datasets/elmy_spot_vs_intraday/processed_data/first_preprocess/" + "X_test_first_preprocess.csv"

In [219]:
X_train.to_csv(pathfile_x_train, index = False)
X_test.to_csv(pathfile_x_test, index = False)